In [1]:
# Add dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - There is no [mac64] chromedriver for browser 91.0.4472 in cache
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/amberteetsel/.wdm/drivers/chromedriver/mac64/91.0.4472.101]


### Visit the NASA Mars News Site

In [3]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
# Scrape to find most recent article title and description
slide_elem.find('div', class_='content_title')

<div class="content_title"><a href="/news/8995/nasa-perseverance-mars-rover-to-acquire-first-sample/" target="_self">NASA Perseverance Mars Rover to Acquire First Sample</a></div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'NASA Perseverance Mars Rover to Acquire First Sample'

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The six-wheeler’s science campaign has laid the groundwork for the mission’s next major milestone.'

### JPL Space Images Featured Image

In [ ]:
# Visit URL
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [ ]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

In [ ]:
# Use the base URL to create an absolute URL
img_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img_url_rel}'
img_url

### Mars Facts

In [ ]:
# Scrape table with Pandas
## Searches for HTML tables and pulls first table it encounters
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df.head()

In [ ]:
# Convert back to HTML-ready code
df.to_html()

### Mars Weather

In [ ]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [ ]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [ ]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [4]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# 2. Create list to hold image links and titles
hemisphere_image_urls = []

# 3. Retrieve full-size image urls and titles for each hemisphere

for hemisphere in range(4):
    # Initialize dictionary to hold title, url
    hemispheres = {}
    # Click link to enhanced image
    browser.find_by_css('a.itemLink.product-item h3')[hemisphere].click()
    html = browser.html
    parsed = soup(html, 'html.parser')
    # Get enhanced image url
    hemispheres['title'] = parsed.find('h2', class_='title').text
    hemispheres['img_url'] = parsed.find('a', text='Sample').get('href')
    hemisphere_image_urls.append(hemispheres)
    # Direct browser back to main page
    browser.back()

In [6]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

### End Automated Browsing

In [7]:
browser.quit()